In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', -1)

import json # library to handle JSON files
import urllib
#!conda install -c conda-forge geopy --yes 
# uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
# uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
%matplotlib inline 
import matplotlib.pyplot as plt



print('Libraries imported.')

Libraries imported.


## Importing data

In [2]:
link1NP=('https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1374388d-1035-4614-b21d-29e7f3cc1053?format=csv')
link2CS=('https://opendata.arcgis.com/datasets/af500b5abb7240399853b35a2362d0c0_0.csv?outSR=%7B%22latestWkid%22%3A26717%2C%22wkid%22%3A26717%7D')

neighprof=pd.read_csv(link1NP)
crimestats=pd.read_csv(link2CS)

link3GD = "https://ckan0.cf.opendata.inter.sandbox-toronto.ca/download_resource/1d02b0f0-d735-4469-8f71-ea6d96b319e4?format=geojson&projection=4326"
response = urllib.request.urlopen(link3GD)
geodata = json.loads(response.read())

link4postal=('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
postal = pd.read_html(link4postal, skiprows=1)[0]

print('Data imported.')

Data imported.


In [3]:
#Creating Neighgeo data
neighborhoods_data = geodata['features']
column_names=['Neighbourhood','Latitude','Longitude']
neighgeo=pd.DataFrame(columns=column_names)

for data in neighborhoods_data: 
    neighborhood_name = data['properties']['AREA_NAME']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = data['properties']['LATITUDE']
    neighborhood_lon = data['properties']['LONGITUDE']
    
    neighgeo = neighgeo.append({'Neighbourhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

                              

Starting data clean up for Neigh Porf

In [4]:
neighprof.shape

(2383, 146)

In [5]:
neighprof.head()

,_id,Category,Topic,Data Source,Characteristic,City of Toronto,Agincourt North,Agincourt South-Malvern West,Alderwood,Annex,Banbury-Don Mills,Bathurst Manor,Bay Street Corridor,Bayview Village,Bayview Woods-Steeles,Bedford Park-Nortown,Beechborough-Greenbrook,Bendale,Birchcliffe-Cliffside,Black Creek,Blake-Jones,Briar Hill-Belgravia,Bridle Path-Sunnybrook-York Mills,Broadview North,Brookhaven-Amesbury,Cabbagetown-South St. James Town,Caledonia-Fairbank,Casa Loma,Centennial Scarborough,Church-Yonge Corridor,Clairlea-Birchmount,Clanton Park,Cliffcrest,Corso Italia-Davenport,Danforth,Danforth East York,Don Valley Village,Dorset Park,Dovercourt-Wallace Emerson-Junction,Downsview-Roding-CFB,Dufferin Grove,East End-Danforth,Edenbridge-Humber Valley,Eglinton East,Elms-Old Rexdale,Englemount-Lawrence,Eringate-Centennial-West Deane,Etobicoke West Mall,Flemingdon Park,Forest Hill North,Forest Hill South,Glenfield-Jane Heights,Greenwood-Coxwell,Guildwood,Henry Farm,High Park North,High Park-Swansea,Highland Creek,Hillcrest Village,Humber Heights-Westmount,Humber Summit,Humbermede,Humewood-Cedarvale,Ionview,Islington-City Centre West,Junction Area,Keelesdale-Eglinton West,Kennedy Park,Kensington-Chinatown,Kingsview Village-The Westway,Kingsway South,Lambton Baby Point,L'Amoreaux,Lansing-Westgate,Lawrence Park North,Lawrence Park South,Leaside-Bennington,Little Portugal,Long Branch,Malvern,Maple Leaf,Markland Wood,Milliken,Mimico (includes Humber Bay Shores),Morningside,Moss Park,Mount Dennis,Mount Olive-Silverstone-Jamestown,Mount Pleasant East,Mount Pleasant West,New Toronto,Newtonbrook East,Newtonbrook West,Niagara,North Riverdale,North St. James Town,Oakridge,Oakwood Village,O'Connor-Parkview,Old East York,Palmerston-Little Italy,Parkwoods-Donalda,Pelmo Park-Humberlea,Playter Estates-Danforth,Pleasant View,Princess-Rosethorn,Regent Park,Rexdale-Kipling,Rockcliffe-Smythe,Roncesvalles,Rosedale-Moore Park,Rouge,Runnymede-Bloor West Village,Rustic,Scarborough Village,South Parkdale,South Riverdale,St.Andrew-Windfields,Steeles,Stonegate-Queensway,Tam O'Shanter-Sullivan,Taylor-Massey,The Beaches,Thistletown-Beaumond Heights,Thorncliffe Park,Trinity-Bellwoods,University,Victoria Village,Waterfront Communities-The Island,West Hill,West Humber-Clairville,Westminster-Branson,Weston,Weston-Pelham Park,Wexford/Maryvale,Willowdale East,Willowdale West,Willowridge-Martingrove-Richview,Woburn,Woodbine Corridor,Woodbine-Lumsden,Wychwood,Yonge-Eglinton,Yonge-St.Clair,York University Heights,Yorkdale-Glen Park
0,1,Neighbourhood Information,Neighbourhood Information,City of Toronto,Neighbourhood Number,NaN,129,128,20,95,42,34,76,52,49,39,112,127,122,24,69,108,41,57,30,71,109,96,133,75,120,33,123,92,66,59,47,126,93,26,83,62,9,138,5,32,11,13,44,102,101,25,65,140,53,88,87,134,48,8,21,22,106,125,14,90,110,124,78,6,15,114,117,38,105,103,56,84,19,132,29,12,130,17,135,73,115,2,99,104,18,50,36,82,68,74,121,107,54,58,80,45,23,67,46,10,72,4,111,86,98,131,89,28,139,85,70,40,116,16,118,61,63,3,55,81,79,43,77,136,1,35,113,91,119,51,37,7,137,64,60,94,100,97,27,31
1,2,Neighbourhood Information,Neighbourhood Information,City of Toronto,TSNS2020 Designation,NaN,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,No Designation,NIA,No Designation,No Designation,No Designation,NIA,NIA,Emerging Neighbourhood,No Designation,No Designation,NIA,No Designation,No Designation,NIA,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,No Designation,Emerging Neighbourhood,NIA,NIA,No Designation,NIA,No Designation,No Designation,NIA,NIA,No 

Clening up the Categories and Topic columns

In [6]:
neighprof['Category'].value_counts()  ##CATEGORY

Language                                   808
Income                                     311
Language of work                           280
Ethnic origin                              279
Education                                  182
Immigration and citizenship                154
Labour                                     76 
Housing                                    74 
Families, households and marital status    70 
Population                                 55 
Aboriginal peoples                         34 
Journey to work                            25 
Mobility                                   18 
Visible minority                           15 
Neighbourhood Information                  2  
Name: Category, dtype: int64

In [7]:
neighprof=neighprof[neighprof.Category != 'Language'] 
neighprof=neighprof[neighprof.Category != 'Language of work'] 
neighprof=neighprof[neighprof.Category != 'Ethnic origin']
neighprof=neighprof[neighprof.Category != 'Immigration and citizenship']
neighprof=neighprof[neighprof.Category != 'Labour']
neighprof=neighprof[neighprof.Category != 'Families, households and marital status']
neighprof=neighprof[neighprof.Category != 'Aboriginal peoples'] 
neighprof=neighprof[neighprof.Category != 'Mobility']
neighprof=neighprof[neighprof.Category != 'Visible minority']
neighprof['Category'].value_counts()

Income                       311
Education                    182
Housing                      74 
Population                   55 
Journey to work              25 
Neighbourhood Information    2  
Name: Category, dtype: int64

In [8]:
neighprof['Topic'].value_counts() ##TOPIC

Major field of study - Classification of Instructional Programs (CIP) 2016                          124
Income sources                                                                                      94 
Income of individuals in 2015                                                                       80 
Income of households in 2015                                                                        59 
Household characteristics                                                                           53 
Age characteristics                                                                                 48 
Income of economic families in 2015                                                                 44 
Highest certificate, diploma or degree                                                              30 
Location of study compared with province or territory of residence with countries outside Canada    28 
Low income in 2015                                              

In [9]:
neighprof=neighprof[neighprof.Topic != 'Income sources'] 
neighprof=neighprof[neighprof.Topic != 'Income of households in 2015'] 
neighprof=neighprof[neighprof.Topic != 'Household characteristics']
neighprof=neighprof[neighprof.Topic != 'Age characteristics']
neighprof=neighprof[neighprof.Topic != 'Income of economic families in 2015']
neighprof=neighprof[neighprof.Topic != 'Location of study compared with province or territory of residence with countries outside Canada']
neighprof=neighprof[neighprof.Topic != 'Low income in 2015'] 
neighprof=neighprof[neighprof.Topic != 'Core housing need']
neighprof=neighprof[neighprof.Topic != 'Income taxes']
neighprof=neighprof[neighprof.Topic != 'Time leaving for work']
neighprof=neighprof[neighprof.Topic != 'Main mode of commuting']
neighprof=neighprof[neighprof.Topic != 'Commuting destination']
#neighprof=neighprof[neighprof.Category != 

neighprof['Topic'].value_counts()

Major field of study - Classification of Instructional Programs (CIP) 2016    124
Income of individuals in 2015                                                 80 
Highest certificate, diploma or degree                                        30 
Population and dwellings                                                      7  
Commuting duration                                                            6  
Neighbourhood Information                                                     2  
Name: Topic, dtype: int64

Transposing the df to get and cont. the clean up

In [10]:
neighprof=neighprof.transpose()
neighprof.head()

,0,1,2,3,4,5,6,7,8,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1969,1970,1971,1972,1973,1974
_id,1,2,3,4,5,6,7,8,9,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1853,1970,1971,1972,1973,1974,1975
Category,Neighbourhood Information,Neighbourhood Information,Population,Population,Population,Population,Population,Population,Population,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Income,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,Education,

Dropping top three rows and setting up Characterisics as axis 0 index

In [11]:
neighprof.drop(['_id','Category', 'Topic','Data Source'], axis=0, inplace=True)

Picking the interesting columns from the dataset

In [12]:
neighprof.head(5)

,0,1,2,3,4,5,6,7,8,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1699,1700,1701,1702,1703,1704,1705,1706,1707,1708,1709,1710,1711,1712,1713,1714,1715,1716,1717,1718,1719,1720,1721,1722,1723,1724,1725,1726,1727,1728,1729,1730,1731,1732,1733,1734,1735,1736,1737,1738,1739,1740,1741,1742,1743,1744,1745,1746,1747,1748,1749,1750,1751,1752,1753,1754,1755,1756,1757,1758,1759,1760,1761,1762,1763,1764,1765,1766,1767,1768,1769,1770,1771,1772,1773,1774,1775,1776,1777,1778,1779,1780,1781,1782,1783,1784,1785,1786,1787,1788,1789,1790,1791,1792,1793,1794,1795,1796,1797,1798,1799,1800,1801,1802,1803,1804,1805,1806,1807,1808,1809,1810,1811,1812,1813,1814,1815,1816,1817,1818,1819,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829,1830,1831,1832,1833,1834,1835,1836,1837,1838,1839,1840,1841,1842,1843,1844,1845,1846,1847,1848,1849,1850,1851,1852,1969,1970,1971,1972,1973,1974
Characteristic,Neighbourhood Number,TSNS2020 Designation,"Population, 2016","Population, 2011",Population Change 2011-2016,Total private dwellings,Private dwellings occupied by usual residents,Population density per square kilometre,Land area in square kilometres,Total - Income statistics in 2015 for the population aged 15 years and over in private households,Number of total income recipients aged 15 years and over in private households,Median total income in 2015 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households - 100% data,Median after-tax income in 2015 among recipients ($),Number of market income recipients aged 15 years and over in private households - 100% data,Median market income in 2015 among recipients ($),Number of government transfers recipients aged 15 years and over in private households - 100% data,Median government transfers in 2015 among recipients ($),Number of employment income recipients aged 15 years and over in private households - 100% data,Median employment income in 2015 among recipients ($),Total - Income statistics in 2015 for the population aged 15 years and over in private households - 25% sample data,Number of total income recipients aged 15 years and over in private households - 25% sample data,Average total income in 2015 among recipients ($),Number of after-tax income recipients aged 15 years and over in private households - 25% sample data,Average after-tax income in 2015 among recipients ($),Number of market income recipients aged 15 years and over in private households - 25% sample data,Average market income in 2015 among recipients ($),Number of government transfers recipients aged 15 years and over in private households - 25% sample data,Average government transfers in 2015 among recipients ($),Number of employment income recipients aged 15 years and over in private households - 25% sample data,Average employment income in 2015 among recipients ($),Total - Employment income statistics for the population aged 15 years and over in private households - 25% sample data,Number of employment income recipients aged 15 years and over in private households who worked full year full time in 2015 - 25% sample data,Median employment income in 2015 for full-year full-time workers ($),Average employment income in 2015 for full-year full-time workers ($),Composition of total income in 2015 of the population aged 15 years and over in private households (%) - 100% data,Market income (%),Employment income (%),Government transfers (%),Total - Total income groups in 2015 for the population aged 15 years and over in private households - 100% data,Without total income,With total income,Percentage with total income,"Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999","$40,000 to $49,99

In [13]:
neigh=neighprof[[0,2,986,987,988,989,990,991,992,993,994,995,996,997,1700,1701,1702,1970,1971,1972,1973,1974]]
neigh.columns = neigh.iloc[0]
neigh.drop(['Characteristic','City of Toronto'], inplace=True)
neigh=neigh.apply((lambda x: x.str.replace(',',''))).astype(float, errors='ignore')
neigh.head()

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


Characteristic,Neighbourhood Number,"Population, 2016","Under $10,000 (including loss)","$10,000 to $19,999","$20,000 to $29,999","$30,000 to $39,999","$40,000 to $49,999","$50,000 to $59,999","$60,000 to $69,999","$70,000 to $79,999","$80,000 and over","$80,000 to $89,999","$90,000 to $99,999","$100,000 and over","No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,"Postsecondary certificate, diploma or degree",Less than 15 minutes,15 to 29 minutes,30 to 44 minutes,45 to 59 minutes,60 minutes and over
Agincourt North,129.0,29113.0,5225.0,6475.0,3945.0,2780.0,2005.0,1220.0,745.0,525.0,720.0,275.0,190.0,245.0,6550.0,7460.0,10985.0,1610.0,3175.0,2940.0,1410.0,2670.0
Agincourt South-Malvern West,128.0,23757.0,4590.0,4640.0,3075.0,2270.0,1665.0,1150.0,695.0,500.0,710.0,290.0,155.0,265.0,4035.0,6090.0,10285.0,1455.0,2735.0,2515.0,1230.0,2230.0
Alderwood,20.0,12054.0,1395.0,1565.0,1590.0,1325.0,1170.0,930.0,625.0,510.0,830.0,350.0,155.0,325.0,2005.0,2960.0,5300.0,1210.0,1815.0,1495.0,630.0,895.0
Annex,95.0,30526.0,3995.0,3790.0,2955.0,2590.0,2370.0,1930.0,1630.0,1215.0,5200.0,870.0,670.0,3660.0,1585.0,4270.0,20430.0,2330.0,6260.0,4000.0,1410.0,905.0
Banbury-Don Mills,42.0,27695.0,3320.0,3265.0,2725.0,2450.0,2360.0,1905.0,1550.0,1135.0,3675.0,905.0,630.0,2150.0,2295.0,5150.0,15950.0,1695.0,3475.0,3110.0,1540.0,1565.0


In [14]:
neigh.columns

Index(['Neighbourhood Number', 'Population, 2016',
       '    Under $10,000 (including loss)', '    $10,000 to $19,999',
       '    $20,000 to $29,999', '    $30,000 to $39,999',
       '    $40,000 to $49,999', '    $50,000 to $59,999',
       '    $60,000 to $69,999', '    $70,000 to $79,999',
       '    $80,000 and over', '      $80,000 to $89,999',
       '      $90,000 to $99,999', '      $100,000 and over',
       '  No certificate, diploma or degree',
       '  Secondary (high) school diploma or equivalency certificate',
       '  Postsecondary certificate, diploma or degree',
       '  Less than 15 minutes', '  15 to 29 minutes', '  30 to 44 minutes',
       '  45 to 59 minutes', '  60 minutes and over'],
      dtype='object', name='Characteristic')

In [15]:
neigh['Low income (less than $19,999) in %']= neigh.iloc[:, 3:5].sum(axis=1)
neigh['Low income (less than $19,999) in %']=neigh['Low income (less than $19,999) in %'].div(neigh['Population, 2016'])*100
neigh['Low-mid income (less than $39,999) in %']= neigh.iloc[:, 5:7].sum(axis=1)
neigh['Low-mid income (less than $39,999) in %']=neigh['Low-mid income (less than $39,999) in %'].div(neigh['Population, 2016'])*100
neigh['Mid-low income (less than $59,999) in %']= neigh.iloc[:, 7:9].sum(axis=1)
neigh['Mid-low income (less than $59,999) in %']=neigh['Mid-low income (less than $59,999) in %'].div(neigh['Population, 2016'])*100
neigh['Mid-high income(less than $79,999) in %']= neigh.iloc[:, 9:11].sum(axis=1)
neigh['Mid-high income(less than $79,999) in %']=neigh['Mid-high income(less than $79,999) in %'].div(neigh['Population, 2016'])*100
neigh['High income ($80,000 and over) in %']= neigh.iloc[:, 11:15].sum(axis=1)
neigh['High income ($80,000 and over) in %']=neigh['High income ($80,000 and over) in %'].div(neigh['Population, 2016'])*100
neigh.drop(['    Under $10,000 (including loss)', '    $10,000 to $19,999', '    $20,000 to $29,999', '    $30,000 to $39,999',       '    $40,000 to $49,999', '    $50,000 to $59,999', '    $60,000 to $69,999', '    $70,000 to $79,999','    $80,000 and over', '      $80,000 to $89,999', '      $90,000 to $99,999', '      $100,000 and over'], axis=1, inplace=True)

neigh.head()

Characteristic,Neighbourhood Number,"Population, 2016","No certificate, diploma or degree",Secondary (high) school diploma or equivalency certificate,"Postsecondary certificate, diploma or degree",Less than 15 minutes,15 to 29 minutes,30 to 44 minutes,45 to 59 minutes,60 minutes and over,"Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %"
Agincourt North,129.0,29113.0,6550.0,7460.0,10985.0,1610.0,3175.0,2940.0,1410.0,2670.0,35.791571,16.435956,6.749562,4.276440,24.937313
Agincourt South-Malvern West,128.0,23757.0,4035.0,6090.0,10285.0,1455.0,2735.0,2515.0,1230.0,2230.0,32.474639,16.563539,7.766132,5.093236,19.973061
Alderwood,20.0,12054.0,2005.0,2960.0,5300.0,1210.0,1815.0,1495.0,630.0,895.0,26.173884,20.698523,12.900282,11.116642,23.519164
Annex,95.0,30526.0,1585.0,4270.0,20430.0,2330.0,6260.0,4000.0,1410.0,905.0,22.095918,16.248444,11.662190,21.014873,22.226954
Banbury-Don Mills,42.0,27695.0,2295.0,5150.0,15950.0,1695.0,3475.0,3110.0,1540.0,1565.0,21.628453,17.367756,12.475176,17.367756,21.592345


In [16]:
neigh['No certificate, diploma or degree in %']=neigh['  No certificate, diploma or degree'].div(neigh['Population, 2016'])*100
neigh['Secondary (high) school diploma or equivalency certificate in %']=neigh['  Secondary (high) school diploma or equivalency certificate'].div(neigh['Population, 2016'])*100
neigh['Postsecondary certificate, diploma or degree in %']=neigh['  Postsecondary certificate, diploma or degree'].div(neigh['Population, 2016'])*100
neigh['Less than 15 minutes in %']=neigh['  Less than 15 minutes'].div(neigh['Population, 2016'])*100
neigh['15 to 29 minutes in %']=neigh['  15 to 29 minutes'].div(neigh['Population, 2016'])*100
neigh['30 to 44 minutes in %']=neigh['  30 to 44 minutes'].div(neigh['Population, 2016'])*100
neigh['45 to 59 minutes in %']=neigh['  45 to 59 minutes'].div(neigh['Population, 2016'])*100
neigh['60 minutes and over in %']=neigh['  60 minutes and over'].div(neigh['Population, 2016'])*100
neigh.drop(['  No certificate, diploma or degree',
       '  Secondary (high) school diploma or equivalency certificate',
       '  Postsecondary certificate, diploma or degree',
       '  Less than 15 minutes', '  15 to 29 minutes', '  30 to 44 minutes',
       '  45 to 59 minutes', '  60 minutes and over',], axis=1, inplace=True)


In [17]:
neigh.shape

(140, 15)

In [18]:
neigh.head()

Characteristic,Neighbourhood Number,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %
Agincourt North,129.0,29113.0,35.791571,16.435956,6.749562,4.276440,24.937313,22.498540,25.624292,37.732285,5.530176,10.905781,10.098581,4.843197,9.171161
Agincourt South-Malvern West,128.0,23757.0,32.474639,16.563539,7.766132,5.093236,19.973061,16.984468,25.634550,43.292503,6.124511,11.512396,10.586353,5.177421,9.386707
Alderwood,20.0,12054.0,26.173884,20.698523,12.900282,11.116642,23.519164,16.633483,24.556164,43.968807,10.038162,15.057242,12.402522,5.226481,7.424921
Annex,95.0,30526.0,22.095918,16.248444,11.662190,21.014873,22.226954,5.192295,13.988076,66.926554,7.632838,20.507109,13.103584,4.619013,2.964686
Banbury-Don Mills,42.0,27695.0,21.628453,17.367756,12.475176,17.367756,21.592345,8.286694,18.595414,57.591623,6.120238,12.547391,11.229464,5.560571,5.650840


In [19]:
neigh=neigh.astype(int, errors='ignore')
neigh.head()

Characteristic,Neighbourhood Number,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %
Agincourt North,129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9
Agincourt South-Malvern West,128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9
Alderwood,20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7
Annex,95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2
Banbury-Don Mills,42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5


In [20]:
neigh.set_index('Neighbourhood Number', inplace=True)
neigh.rename_axis(mapper='' ,axis=1, inplace=True)

In [21]:
neigh.head()

,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %
Neighbourhood Number,,,,,,,,,,,,,,
129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9
128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9
20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7
95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2
42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5


Starting data clean up for Crime Stats

In [22]:
crimestats.shape

(140, 56)

In [23]:
crimestats.head()

,OBJECTID,Neighbourhood_Crime_Rates_Neigh,Neighbourhood_Crime_Rates_Hood_,Hood_ID,Neighbourhood,Assault_2014,Assault_2015,Assault_2016,Assault_2017,Assault_2018,Assault_AVG,Assault_CHG,Assault_Rate_2018,Auto_Theft_2014,Auto_Theft_2015,Auto_Theft_2016,Auto_Theft_2017,Auto_Theft_2018,AutoTheft_AVG,AutoTheft_CHG,AutoTheft_Rate_2018,BreakandEnter_2014,BreakandEnter_2015,BreakandEnter_2016,BreakandEnter_2017,BreakandEnter_2018,BreakandEnter_AVG,BreakandEnter_CHG,BreakandEnter_Rate_2018,Robbery_2014,Robbery_2015,Robbery_2016,Robbery_2017,Robbery_2018,Robbery_AVG,Robbery_CHG,Robbery_Rate_2018,Theft_Over_2014,Theft_Over_2015,Theft_Over_2016,Theft_Over_2017,Theft_Over_2018,TheftOver_AVG,TheftOver_CHG,TheftOver_Rate_2018,Homicide_2014,Homicide_2015,Homicide_2016,Homicide_2017,Homicide_2018,Homicide_AVG,Homicide_CHG,Homicide_Rate_2018,Population,Shape__Area,Shape__Length
0,1,Yonge-St.Clair,97,97,Yonge-St.Clair,58,38,51,46,61,50.8,33%,1912.8,28,32,22,46,69,39.4,50%,2163.7,29,16,28,32,23,25.6,-28%,721.2,12,25.0,14,21,19,18.2,-10%,595.8,3,6,4.0,6,3.0,4.4,-50%,94.1,0,0,0,0,0,NaN,NaN,0.0,3189,1.161315e+06,5873.270507
1,2,York University Heights,27,27,York University Heights,78,101,111,120,138,109.6,15%,375.4,16,13,8,19,23,15.8,21%,62.6,43,59,61,34,52,49.8,53%,141.4,28,22.0,23,31,15,23.8,-52%,40.8,4,4,3.0,6,2.0,3.8,-67%,5.4,1,0,2,0,1,1.3,NaN,2.7,36764,1.324666e+07,18504.777616
2,3,Lansing-Westgate,38,38,Lansing-Westgate,216,203,223,226,197,213.0,-13%,1923.5,21,25,23,36,22,25.4,-39%,214.8,43,78,72,73,52,63.6,-29%,507.7,81,53.0,49,43,41,53.4,-5%,400.3,8,4,7.0,2,6.0,5.4,200%,58.6,0,0,0,0,0,NaN,NaN,0.0,10242,5.346186e+06,11112.109419
3,4,Yorkdale-Glen Park,31,31,Yorkdale-Glen Park,121,141,136,124,127,129.8,2%,696.5,47,27,25,30,28,31.4,-7%,153.6,83,50,65,31,56,57.0,81%,307.1,42,32.0,22,18,35,29.8,94%,192.0,8,6,8.0,10,13.0,9.0,30%,71.3,1,1,1,1,2,1.2,100%,11.0,18233,6.038326e+06,10079.426837
4,5,Stonegate-Queensway,16,16,Stonegate-Queensway,109,140,124,112,128,122.6,14%,576.4,20,17,22,14,41,22.8,193%,184.6,48,37,34,37,41,39.4,11%,184.6,33,22.0,41,35,36,33.4,3%,162.1,10,3,4.0,5,4.0,5.2,-20%,18.0,1,0,0,0,0,1.0,NaN,0.0,22207,7.946202e+06,11853.189803


Changing 

In [24]:
crimestats.columns

Index(['OBJECTID', 'Neighbourhood_Crime_Rates_Neigh',
       'Neighbourhood_Crime_Rates_Hood_', 'Hood_ID', 'Neighbourhood',
       'Assault_2014', 'Assault_2015', 'Assault_2016', 'Assault_2017',
       'Assault_2018', 'Assault_AVG', 'Assault_CHG', 'Assault_Rate_2018',
       'Auto_Theft_2014', 'Auto_Theft_2015', 'Auto_Theft_2016',
       'Auto_Theft_2017', 'Auto_Theft_2018', 'AutoTheft_AVG', 'AutoTheft_CHG',
       'AutoTheft_Rate_2018', 'BreakandEnter_2014', 'BreakandEnter_2015',
       'BreakandEnter_2016', 'BreakandEnter_2017', 'BreakandEnter_2018',
       'BreakandEnter_AVG', 'BreakandEnter_CHG', 'BreakandEnter_Rate_2018',
       'Robbery_2014', 'Robbery_2015', 'Robbery_2016', 'Robbery_2017',
       'Robbery_2018', 'Robbery_AVG', 'Robbery_CHG', 'Robbery_Rate_2018',
       'Theft_Over_2014', 'Theft_Over_2015', 'Theft_Over_2016',
       'Theft_Over_2017', 'Theft_Over_2018', 'TheftOver_AVG', 'TheftOver_CHG',
       'TheftOver_Rate_2018', 'Homicide_2014', 'Homicide_2015',
       'Homic

In [25]:
crime=crimestats.drop(['OBJECTID', 'Neighbourhood_Crime_Rates_Neigh', 'Neighbourhood_Crime_Rates_Hood_', 'Assault_2014', 'Assault_2015', 'Assault_2016', 'Assault_2017','Assault_AVG', 'Assault_CHG', 'Assault_Rate_2018', 'Auto_Theft_2014', 'Auto_Theft_2015', 'Auto_Theft_2016','Auto_Theft_2017', 'AutoTheft_AVG', 'AutoTheft_CHG',  'AutoTheft_Rate_2018', 'BreakandEnter_2014', 'BreakandEnter_2015',     'BreakandEnter_2016', 'BreakandEnter_2017',     'BreakandEnter_AVG', 'BreakandEnter_CHG', 'BreakandEnter_Rate_2018',  'Robbery_2014', 'Robbery_2015', 'Robbery_2016', 'Robbery_2017',  'Robbery_AVG', 'Robbery_CHG', 'Robbery_Rate_2018',  'Theft_Over_2014', 'Theft_Over_2015', 'Theft_Over_2016',    'Theft_Over_2017', 'TheftOver_AVG', 'TheftOver_CHG',     'TheftOver_Rate_2018', 'Homicide_2014', 'Homicide_2015',    'Homicide_2016', 'Homicide_2017', 'Homicide_AVG',     'Homicide_CHG', 'Homicide_Rate_2018', 'Population', 'Shape__Area','Shape__Length'], axis=1)
crimestats.astype(int, errors='ignore', inplace=True)
crime.set_index('Hood_ID', inplace=True)
crime.head()


,Neighbourhood,Assault_2018,Auto_Theft_2018,BreakandEnter_2018,Robbery_2018,Theft_Over_2018,Homicide_2018
Hood_ID,,,,,,,
97,Yonge-St.Clair,61,69,23,19,3.0,0
27,York University Heights,138,23,52,15,2.0,1
38,Lansing-Westgate,197,22,52,41,6.0,0
31,Yorkdale-Glen Park,127,28,56,35,13.0,2
16,Stonegate-Queensway,128,41,41,36,4.0,0


In [26]:
crime.columns

Index(['Neighbourhood', 'Assault_2018', 'Auto_Theft_2018',
       'BreakandEnter_2018', 'Robbery_2018', 'Theft_Over_2018',
       'Homicide_2018'],
      dtype='object')

In [27]:
crime['Total Crime per pop']=crime.iloc[:, 1:7].sum(axis=1).astype(int, errors='ignore')
crime.drop(['Assault_2018', 'Auto_Theft_2018',
       'BreakandEnter_2018', 'Robbery_2018', 'Theft_Over_2018',
       'Homicide_2018'], axis=1, inplace=True)
crime.head()

,Neighbourhood,Total Crime per pop
Hood_ID,,
97,Yonge-St.Clair,175
27,York University Heights,231
38,Lansing-Westgate,318
31,Yorkdale-Glen Park,261
16,Stonegate-Queensway,250


Joining both datasets

In [28]:
df=neigh.join(crime)
#df.columns=['Population, 2016', 'No certificate, diploma or degree', 'Secondary (high) school diploma or equivalency certificate','Postsecondary certificate, diploma or degree',      'Less than 15 minutes', '15 to 29 minutes', '30 to 44 minutes',       '45 to 59 minutes', '60 minutes and over',       'Low income (less than $19,999)', 'Low-mid income (less than $39,999)',       'Mid-low income (less than $59,999)',       'Mid-high income(less than $79,999)', 'High income ($80,000 and over)','Neighbourhood', 'Assault_2018', 'Auto_Theft_2018','BreakandEnter_2018', 'Robbery_2018', 'Theft_Over_2018','Homicide_2018', 'Total Crime']
df['Total Crime per pop']=df['Total Crime per pop'].div(df['Population, 2016'])

df.head()

,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Neighbourhood,Total Crime per pop
Neighbourhood Number,,,,,,,,,,,,,,,,
129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9,Agincourt North,0.017621
128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9,Agincourt South-Malvern West,0.006482
20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7,Alderwood,0.018500
95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2,Annex,0.005045
42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5,Banbury-Don Mills,0.002925


In [29]:
df.reset_index(inplace=True)
df.head()

,Neighbourhood Number,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Neighbourhood,Total Crime per pop
0,129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9,Agincourt North,0.017621
1,128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9,Agincourt South-Malvern West,0.006482
2,20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7,Alderwood,0.018500
3,95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2,Annex,0.005045
4,42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5,Banbury-Don Mills,0.002925


Adding Postal Code information

In [30]:
postal.columns=["PostalCode","Borough","Neighbourhood"]
postal.drop(['Borough'],axis=1, inplace=True)
postal.head()

,PostalCode,Neighbourhood
0,M2A,Not assigned
1,M3A,Parkwoods
2,M4A,Victoria Village
3,M5A,Harbourfront
4,M5A,Regent Park


In [31]:
postal = postal[postal.Neighbourhood !='Not assigned'].reset_index(drop=True)
postal.head()

,PostalCode,Neighbourhood
0,M3A,Parkwoods
1,M4A,Victoria Village
2,M5A,Harbourfront
3,M5A,Regent Park
4,M6A,Lawrence Heights


## Getting data from Forsquare

In [32]:
CLIENT_ID = 'PE2TL1SIA13UAMEYUJT4T5AVSKTEVMVIJICCP1V3W2AXQFMJ' # your Foursquare ID
CLIENT_SECRET = '5H5PKRONFUVAZWYICLXZIYOLEHXUO5ZIHVGFMKH5Z5MV0LTA' # your Foursquare Secret
VERSION = '20180604'
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PE2TL1SIA13UAMEYUJT4T5AVSKTEVMVIJICCP1V3W2AXQFMJ
CLIENT_SECRET:5H5PKRONFUVAZWYICLXZIYOLEHXUO5ZIHVGFMKH5Z5MV0LTA


In [33]:
search_query = 'vegetarian'
print(search_query + ' .... OK!')
categoryid='4bf58dd8d48988d1d3941735'
near='Toronto, ON'

vegetarian .... OK!


In [34]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&categoryId={}&near={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, categoryid, near)
url

'https://api.foursquare.com/v2/venues/search?client_id=PE2TL1SIA13UAMEYUJT4T5AVSKTEVMVIJICCP1V3W2AXQFMJ&client_secret=5H5PKRONFUVAZWYICLXZIYOLEHXUO5ZIHVGFMKH5Z5MV0LTA&v=20180604&categoryId=4bf58dd8d48988d1d3941735&near=Toronto, ON'

In [35]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d1a1725f19f4400258359b6'},
 'response': {'venues': [{'id': '5bd1fc9bfdb9a7002c331d1c',
    'name': 'Mad Radish',
    'location': {'address': '2293 Yonge Street',
     'lat': 43.707672,
     'lng': -79.398424,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.707672,
       'lng': -79.398424}],
     'postalCode': 'M4P',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['2293 Yonge Street', 'Toronto ON M4P', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d1d3941735',
      'name': 'Vegetarian / Vegan Restaurant',
      'pluralName': 'Vegetarian / Vegan Restaurants',
      'shortName': 'Vegetarian / Vegan',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1561990949',
    'hasPerk': False},
   {'id': '5c9d602f25fb7b002c79b669',
    'name': 'Planta Queen',
    'locati

In [36]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()



,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}]",False,5bd1fc9bfdb9a7002c331d1c,2293 Yonge Street,CA,Toronto,Canada,NaN,"[2293 Yonge Street, Toronto ON M4P, Canada]","[{'label': 'display', 'lat': 43.707672, 'lng': -79.398424}]",43.707672,-79.398424,NaN,M4P,ON,Mad Radish,v-1561990949,NaN
1,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}]",False,5c9d602f25fb7b002c79b669,180 Queen St W,CA,Toronto,Canada,NaN,"[180 Queen St W, Toronto ON M5V 3K1, Canada]","[{'label': 'display', 'lat': 43.650622, 'lng': -79.388154}]",43.650622,-79.388154,NaN,M5V 3K1,ON,Planta Queen,v-1561990949,NaN
2,"[{'id': '4bf58dd8d48988d1c5941735', 'name': 'Sandwich Place', 'pluralName': 'Sandwich Places', 'shortName': 'Sandwiches', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/deli_', 'suffix': '.png'}, 'primary': True}]",False,4b9a86fcf964a52008be35e3,1975 Kennedy Road,CA,Scarborough,Canada,NaN,"[1975 Kennedy Road, Scarborough ON M1P 2M1, Canada]","[{'label': 'display', 'lat': 43.76990435997661, 'lng': -79.2818319251802}]",43.769904,-79.281832,NaN,M1P 2M1,ON,Pita Pit,v-1561990949,NaN
3,"[{'id': '52f2ab2ebcbc57f1066b8b41', 'name': 'Smoothie Shop', 'pluralName': 'Smoothie Shops', 'shortName': 'Smoothie Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/juicebar_', 'suffix': '.png'}, 'primary': True}]",False,5a0472ea6e465038158bf043,613 King Street West,CA,Toronto,Canada,NaN,"[613 King Street West, Toronto ON M5V 0G9, Canada]","[{'label': 'display', 'lat': 43.644085, 'lng': -79.401444}]",43.644085,-79.401444,NaN,M5V 0G9,ON,iQ Food Co.,v-1561990949,NaN
4,"[{'id': '4bf58dd8d48988d1d3941735', 'name': 'Vegetarian / Vegan Restaurant', 'pluralName': 'Vegetarian / Vegan Restaurants', 'shortName': 'Vegetarian / Vegan', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/vegetarian_', 'suffix': '.png'}, 'primary': True}]",False,4ad4c05cf964a5200ff620e3,147 Spadina Ave.,CA,Toronto,Canada,at Richmond St. W.,"[147 Spadina Ave. (at Richmond St. W.), Toronto ON M5V 2L7, Canada]","[{'label': 'display', 'lat': 43.648048207763615, 'lng': -79.39600791264331}]",43.648048,-79.396008,NaN,M5V 2L7,ON,Fresh On Spadina,v-1561990949,NaN


In [37]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]
dataframe_filtered.head()

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Mad Radish,Vegetarian / Vegan Restaurant,2293 Yonge Street,CA,Toronto,Canada,NaN,"[2293 Yonge Street, Toronto ON M4P, Canada]","[{'label': 'display', 'lat': 43.707672, 'lng': -79.398424}]",43.707672,-79.398424,NaN,M4P,ON,5bd1fc9bfdb9a7002c331d1c
1,Planta Queen,Vegetarian / Vegan Restaurant,180 Queen St W,CA,Toronto,Canada,NaN,"[180 Queen St W, Toronto ON M5V 3K1, Canada]","[{'label': 'display', 'lat': 43.650622, 'lng': -79.388154}]",43.650622,-79.388154,NaN,M5V 3K1,ON,5c9d602f25fb7b002c79b669
2,Pita Pit,Sandwich Place,1975 Kennedy Road,CA,Scarborough,Canada,NaN,"[1975 Kennedy Road, Scarborough ON M1P 2M1, Canada]","[{'label': 'display', 'lat': 43.76990435997661, 'lng': -79.2818319251802}]",43.769904,-79.281832,NaN,M1P 2M1,ON,4b9a86fcf964a52008be35e3
3,iQ Food Co.,Smoothie Shop,613 King Street West,CA,Toronto,Canada,NaN,"[613 King Street West, Toronto ON M5V 0G9, Canada]","[{'label': 'display', 'lat': 43.644085, 'lng': -79.401444}]",43.644085,-79.401444,NaN,M5V 0G9,ON,5a0472ea6e465038158bf043
4,Fresh On Spadina,Vegetarian / Vegan Restaurant,147 Spadina Ave.,CA,Toronto,Canada,at Richmond St. W.,"[147 Spadina Ave. (at Richmond St. W.), Toronto ON M5V 2L7, Canada]","[{'label': 'display', 'lat': 43.648048207763615, 'lng': -79.39600791264331}]",43.648048,-79.396008,NaN,M5V 2L7,ON,4ad4c05cf964a5200ff620e3


In [38]:
##Cleaning up the df to include only "pure" Vegetarian/Vegan options

dataframe_filtered=dataframe_filtered[dataframe_filtered.categories == 'Vegetarian / Vegan Restaurant']


In [39]:
dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Mad Radish,Vegetarian / Vegan Restaurant,2293 Yonge Street,CA,Toronto,Canada,NaN,"[2293 Yonge Street, Toronto ON M4P, Canada]","[{'label': 'display', 'lat': 43.707672, 'lng': -79.398424}]",43.707672,-79.398424,NaN,M4P,ON,5bd1fc9bfdb9a7002c331d1c
1,Planta Queen,Vegetarian / Vegan Restaurant,180 Queen St W,CA,Toronto,Canada,NaN,"[180 Queen St W, Toronto ON M5V 3K1, Canada]","[{'label': 'display', 'lat': 43.650622, 'lng': -79.388154}]",43.650622,-79.388154,NaN,M5V 3K1,ON,5c9d602f25fb7b002c79b669
4,Fresh On Spadina,Vegetarian / Vegan Restaurant,147 Spadina Ave.,CA,Toronto,Canada,at Richmond St. W.,"[147 Spadina Ave. (at Richmond St. W.), Toronto ON M5V 2L7, Canada]","[{'label': 'display', 'lat': 43.648048207763615, 'lng': -79.39600791264331}]",43.648048,-79.396008,NaN,M5V 2L7,ON,4ad4c05cf964a5200ff620e3
5,Planta Yorkville,Vegetarian / Vegan Restaurant,1221 Bay Street,CA,Toronto,Canada,NaN,"[1221 Bay Street, Toronto ON M5R 3P5, Canada]","[{'label': 'display', 'lat': 43.670213457162674, 'lng': -79.38951245082394}]",43.670213,-79.389512,NaN,M5R 3P5,ON,57e074a7498ef24d3980d2f5
6,Rosalinda,Vegetarian / Vegan Restaurant,133 Richmond St W,CA,Toronto,Canada,NaN,"[133 Richmond St W, Toronto ON M5H 2L2, Canada]","[{'label': 'display', 'lat': 43.650252483069295, 'lng': -79.38515575620075}]",43.650252,-79.385156,NaN,M5H 2L2,ON,5aff06ca6e4650002cc6286b
8,Hello 123,Vegetarian / Vegan Restaurant,1122 Queen St W,CA,Toronto,Canada,NaN,"[1122 Queen St W, Toronto ON M6J 1H9, Canada]","[{'label': 'display', 'lat': 43.64326365057558, 'lng': -79.42351572944511}]",43.643264,-79.423516,NaN,M6J 1H9,ON,59d03b76d1a4022c64b036f8
9,Lotus Pond Vegetarian Restaurant 蓮花素食,Vegetarian / Vegan Restaurant,3838 Midland Ave.,CA,Scarborough,Canada,btwn Passmore & McNicoll Ave.,"[3838 Midland Ave. (btwn Passmore & McNicoll Ave.), Scarborough ON M1K 5V5, Canada]","[{'label': 'display', 'lat': 43.819420734434544, 'lng': -79.294681915014}]",43.819421,-79.294682,NaN,M1K 5V5,ON,4b78354df964a52070bc2ee3
11,Fresh on Bloor,Vegetarian / Vegan Restaurant,326 Bloor St. W,CA,Toronto,Canada,NaN,"[326 Bloor St. W, Toronto ON M5S 1W5, Canada]","[{'label': 'display', 'lat': 43.66675488472059, 'lng': -79.40349130034014}]",43.666755,-79.403491,NaN,M5S 1W5,ON,4df15e3c1f6e818dadf7afc0
12,Fresh On Front,Vegetarian / Vegan Restaurant,47 Front Street E,CA,Toronto,Canada,Church Street,"[47 Front Street E (Church Street), Toronto ON M5E 1B3, Canada]","[{'label': 'display', 'lat': 43.64781466405137, 'lng': -79.3744527618627}]",43.647815,-79.374453,NaN,M5E 1B3,ON,5b5bca904aa3f8002c97f85d
14,Kupfert & Kim,Vegetarian / Vegan Restaurant,140 Spadina Ave,CA,Toronto,Canada,at Richmond St W,"[140 Spadina Ave (at Richmond St W), Toronto ON M5V 2L4, Canada]","[{'label': 'display', 'lat': 43.64794381005563, 'lng': -79.39612137378384}]",43.647944,-79.396121,NaN,M5V 2L4,ON,5632af4b498ee60496a2d7db


Adding Neighbourhoods to the venues df


In [40]:
dataframe_filtered['postalCode']=dataframe_filtered.postalCode.str.split(' ', expand=True)
venuspostal=dataframe_filtered.merge(postal, left_on='postalCode', right_on='PostalCode')
venuspostal.drop(['neighborhood','postalCode'], axis=1, inplace=True)
venuspostal.head()

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,state,id,PostalCode,Neighbourhood
0,Mad Radish,Vegetarian / Vegan Restaurant,2293 Yonge Street,CA,Toronto,Canada,NaN,"[2293 Yonge Street, Toronto ON M4P, Canada]","[{'label': 'display', 'lat': 43.707672, 'lng': -79.398424}]",43.707672,-79.398424,ON,5bd1fc9bfdb9a7002c331d1c,M4P,Davisville North
1,Fresh,Vegetarian / Vegan Restaurant,90 Eglinton Avenue East,CA,Toronto,Canada,Yonge & Eglinton,"[90 Eglinton Avenue East (Yonge & Eglinton), Toronto ON M4P 1A6, Canada]","[{'label': 'display', 'lat': 43.707324410453595, 'lng': -79.39564918411965}]",43.707324,-79.395649,ON,521e0c6c04939a8ad55d93d3,M4P,Davisville North
2,Planta Queen,Vegetarian / Vegan Restaurant,180 Queen St W,CA,Toronto,Canada,NaN,"[180 Queen St W, Toronto ON M5V 3K1, Canada]","[{'label': 'display', 'lat': 43.650622, 'lng': -79.388154}]",43.650622,-79.388154,ON,5c9d602f25fb7b002c79b669,M5V,CN Tower
3,Planta Queen,Vegetarian / Vegan Restaurant,180 Queen St W,CA,Toronto,Canada,NaN,"[180 Queen St W, Toronto ON M5V 3K1, Canada]","[{'label': 'display', 'lat': 43.650622, 'lng': -79.388154}]",43.650622,-79.388154,ON,5c9d602f25fb7b002c79b669,M5V,Bathurst Quay
4,Planta Queen,Vegetarian / Vegan Restaurant,180 Queen St W,CA,Toronto,Canada,NaN,"[180 Queen St W, Toronto ON M5V 3K1, Canada]","[{'label': 'display', 'lat': 43.650622, 'lng': -79.388154}]",43.650622,-79.388154,ON,5c9d602f25fb7b002c79b669,M5V,Island airport


In [41]:
venuspostal=venuspostal.drop_duplicates('name', keep='first').sort_values(by=['PostalCode']).reset_index(drop=True)
venuspostal.head()

,name,categories,address,cc,city,country,crossStreet,formattedAddress,labeledLatLngs,lat,lng,state,id,PostalCode,Neighbourhood
0,Lotus Pond Vegetarian Restaurant 蓮花素食,Vegetarian / Vegan Restaurant,3838 Midland Ave.,CA,Scarborough,Canada,btwn Passmore & McNicoll Ave.,"[3838 Midland Ave. (btwn Passmore & McNicoll Ave.), Scarborough ON M1K 5V5, Canada]","[{'label': 'display', 'lat': 43.819420734434544, 'lng': -79.294681915014}]",43.819421,-79.294682,ON,4b78354df964a52070bc2ee3,M1K,East Birchmount Park
1,Mad Radish,Vegetarian / Vegan Restaurant,2293 Yonge Street,CA,Toronto,Canada,NaN,"[2293 Yonge Street, Toronto ON M4P, Canada]","[{'label': 'display', 'lat': 43.707672, 'lng': -79.398424}]",43.707672,-79.398424,ON,5bd1fc9bfdb9a7002c331d1c,M4P,Davisville North
2,Fresh,Vegetarian / Vegan Restaurant,90 Eglinton Avenue East,CA,Toronto,Canada,Yonge & Eglinton,"[90 Eglinton Avenue East (Yonge & Eglinton), Toronto ON M4P 1A6, Canada]","[{'label': 'display', 'lat': 43.707324410453595, 'lng': -79.39564918411965}]",43.707324,-79.395649,ON,521e0c6c04939a8ad55d93d3,M4P,Davisville North
3,nutbar,Vegetarian / Vegan Restaurant,1240 Yonge St,CA,Toronto,Canada,NaN,"[1240 Yonge St, Toronto ON M4T 1W5, Canada]","[{'label': 'display', 'lat': 43.682810954698965, 'lng': -79.39194466929969}]",43.682811,-79.391945,ON,5890f70e65be581749d7522e,M4T,Moore Park
4,Vegan Bear,Vegetarian / Vegan Restaurant,NaN,CA,Toronto,Canada,NaN,"[Toronto ON M5B 1R7, Canada]","[{'label': 'display', 'lat': 43.657124, 'lng': -79.380948}]",43.657124,-79.380948,ON,5cf43e873ba767002c95ce10,M5B,Ryerson


In [42]:
venuescount=venuspostal.groupby(['Neighbourhood']).count().sort_values(by=['name'])
venuescount=venuescount['name']

In [43]:
df1=df


In [44]:
df1.head()

,Neighbourhood Number,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Neighbourhood,Total Crime per pop
0,129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9,Agincourt North,0.017621
1,128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9,Agincourt South-Malvern West,0.006482
2,20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7,Alderwood,0.018500
3,95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2,Annex,0.005045
4,42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5,Banbury-Don Mills,0.002925


In [45]:
venneigh={'Neighbourhood': ['Waterfront Communities-The Island', 'Church-Yonge Corridor','Willowdale West','Bay Street Corridor','Yonge-St.Clair','Little Portugal','Mount Pleasant West','Kensington-Chinatown','University','Annex' ], 'Number of veg venu': [3, 1, 1, 1, 1, 1, 2, 2, 2, 3]}
venneighdf = pd.DataFrame(data=venneigh)
venhas={'Neighbourhood': ['Waterfront Communities-The Island', 'Church-Yonge Corridor','Willowdale West','Bay Street Corridor','Yonge-St.Clair','Little Portugal','Mount Pleasant West','Kensington-Chinatown','University','Annex' ], 'Has veg venue?': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
venhashdf = pd.DataFrame(data=venhas)

In [46]:
df1=df1.merge(venneighdf, on='Neighbourhood', how='outer' ).fillna(0)
df1=df1.merge(venhashdf, on='Neighbourhood', how='outer' ).fillna(0)

In [47]:
df1.head()

,Neighbourhood Number,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Neighbourhood,Total Crime per pop,Number of veg venu,Has veg venue?
0,129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9,Agincourt North,0.017621,0.0,0.0
1,128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9,Agincourt South-Malvern West,0.006482,0.0,0.0
2,20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7,Alderwood,0.018500,0.0,0.0
3,95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2,Annex,0.005045,3.0,1.0
4,42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5,Banbury-Don Mills,0.002925,0.0,0.0


K-means

In [48]:
df2=df1.drop(['Neighbourhood Number'], axis=1).set_index('Neighbourhood')
df2=df2.drop(['Has veg venue?'], axis=1)

In [49]:
df2.head()

,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Total Crime per pop,Number of veg venu
Neighbourhood,,,,,,,,,,,,,,,,
Agincourt North,29113,35,16,6,4,24,22,25,37,5,10,10,4,9,0.017621,0.0
Agincourt South-Malvern West,23757,32,16,7,5,19,16,25,43,6,11,10,5,9,0.006482,0.0
Alderwood,12054,26,20,12,11,23,16,24,43,10,15,12,5,7,0.018500,0.0
Annex,30526,22,16,11,21,22,5,13,66,7,20,13,4,2,0.005045,3.0
Banbury-Don Mills,27695,21,17,12,17,21,8,18,57,6,12,11,5,5,0.002925,0.0


In [50]:
from sklearn.preprocessing import StandardScaler
X = df2.values[:,1:]
X = np.nan_to_num(X)
cluster_dataset = StandardScaler().fit_transform(X)
cluster_dataset

array([[ 1.42198501, -0.30676726, -1.28626871, ...,  0.92308689,
         0.10559032, -0.25035217],
       [ 0.89016515, -0.30676726, -0.88698352, ...,  0.92308689,
        -0.64058148, -0.25035217],
       [-0.17347457,  1.37744517,  1.10944241, ...,  0.13025152,
         0.16447975, -0.25035217],
       ...,
       [-1.59166087,  0.53533895,  1.5087276 , ..., -1.45541921,
        -0.13907313,  1.81137158],
       [ 0.89016515,  0.95639206, -0.88698352, ...,  1.71592226,
        -0.51401293, -0.25035217],
       [ 0.35834529,  0.95639206, -0.48769833, ..., -0.66258384,
         0.10621842, -0.25035217]])

In [65]:
num_clusters = 3

k_means = KMeans(init="k-means++", n_clusters=num_clusters, n_init=15)
k_means.fit(cluster_dataset)
labels = k_means.labels_

print(labels)

[0 0 0 2 1 0 2 1 1 1 0 0 0 0 1 0 1 0 0 2 0 1 0 2 0 1 0 0 1 1 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 1 0 1 1 0 0 2 0 1 1 0 1 1 1 1 2 0 0
 0 1 0 1 0 2 0 0 1 2 0 1 0 2 1 0 0 0 0 1 2 0 0 1 0 1 0 0 0 1 1 0 1 0 0 0 1
 1 0 1 0 0 1 0 0 1 2 0 2 0 0 0 0 0 0 1 1 0 0 1 1 1 1 2 0 0]


In [66]:
df1["Labels"] = labels
df2["Labels"] = labels
df1.head()

,Neighbourhood Number,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Neighbourhood,Total Crime per pop,Number of veg venu,Has veg venue?,Labels
0,129,29113,35,16,6,4,24,22,25,37,5,10,10,4,9,Agincourt North,0.017621,0.0,0.0,0
1,128,23757,32,16,7,5,19,16,25,43,6,11,10,5,9,Agincourt South-Malvern West,0.006482,0.0,0.0,0
2,20,12054,26,20,12,11,23,16,24,43,10,15,12,5,7,Alderwood,0.018500,0.0,0.0,0
3,95,30526,22,16,11,21,22,5,13,66,7,20,13,4,2,Annex,0.005045,3.0,1.0,2
4,42,27695,21,17,12,17,21,8,18,57,6,12,11,5,5,Banbury-Don Mills,0.002925,0.0,0.0,1


In [67]:
df2.groupby('Labels').mean()

,"Population, 2016","Low income (less than $19,999) in %","Low-mid income (less than $39,999) in %","Mid-low income (less than $59,999) in %","Mid-high income(less than $79,999) in %","High income ($80,000 and over) in %","No certificate, diploma or degree in %",Secondary (high) school diploma or equivalency certificate in %,"Postsecondary certificate, diploma or degree in %",Less than 15 minutes in %,15 to 29 minutes in %,30 to 44 minutes in %,45 to 59 minutes in %,60 minutes and over in %,Total Crime per pop,Number of veg venu
Labels,,,,,,,,,,,,,,,,
0,20264.607595,30.569620,17.670886,7.936709,5.430380,20.987342,17.379747,22.556962,41.430380,4.430380,10.797468,11.594937,6.101266,8.379747,0.016443,0.000000
1,17012.770833,21.770833,14.979167,10.437500,18.333333,23.541667,8.854167,16.312500,56.708333,4.833333,12.666667,14.729167,6.833333,4.583333,0.015355,0.020833
2,24158.000000,24.384615,17.461538,12.538462,16.076923,18.615385,6.615385,16.153846,64.384615,8.461538,19.846154,14.538462,4.692308,4.000000,0.016170,1.230769


Clustered DB (with names)

In [68]:
clusteredmd=df1.replace({'Labels' : { 2 : 'GoodSpot', 1 : 'OKspot', 0 : 'BadSpot' }})

In [69]:
clusteredmd.columns

Index(['Neighbourhood Number', 'Population, 2016',
       'Low income (less than $19,999) in %',
       'Low-mid income (less than $39,999) in %',
       'Mid-low income (less than $59,999) in %',
       'Mid-high income(less than $79,999) in %',
       'High income ($80,000 and over) in %',
       'No certificate, diploma or degree in %',
       'Secondary (high) school diploma or equivalency certificate in %',
       'Postsecondary certificate, diploma or degree in %',
       'Less than 15 minutes in %', '15 to 29 minutes in %',
       '30 to 44 minutes in %', '45 to 59 minutes in %',
       '60 minutes and over in %', 'Neighbourhood', 'Total Crime per pop',
       'Number of veg venu', 'Has veg venue?', 'Labels'],
      dtype='object')

In [70]:
clusteredmd.drop(['Neighbourhood Number', 'Population, 2016',
       'Low income (less than $19,999) in %',
       'Low-mid income (less than $39,999) in %',
       'Mid-low income (less than $59,999) in %',
       'Mid-high income(less than $79,999) in %',
       'High income ($80,000 and over) in %',
       'No certificate, diploma or degree in %',
       'Secondary (high) school diploma or equivalency certificate in %',
       'Postsecondary certificate, diploma or degree in %',
       'Less than 15 minutes in %', '15 to 29 minutes in %',
       '30 to 44 minutes in %', '45 to 59 minutes in %',
       '60 minutes and over in %', 'Total Crime per pop'], axis=1, inplace=True)

In [71]:
clusteredmd

,Neighbourhood,Number of veg venu,Has veg venue?,Labels
0,Agincourt North,0.0,0.0,BadSpot
1,Agincourt South-Malvern West,0.0,0.0,BadSpot
2,Alderwood,0.0,0.0,BadSpot
3,Annex,3.0,1.0,GoodSpot
4,Banbury-Don Mills,0.0,0.0,OKspot
5,Bathurst Manor,0.0,0.0,BadSpot
6,Bay Street Corridor,1.0,1.0,GoodSpot
7,Bayview Village,0.0,0.0,OKspot
8,Bayview Woods-Steeles,0.0,0.0,OKspot
9,Bedford Park-Nortown,0.0,0.0,OKspot


In [73]:
vegan_map = folium.Map(location=[43.653963, -79.387207], zoom_start=11) 
# add the Vegan restaurants as blue circle markers
for lat, lng, label in zip(venuspostal.lat, venuspostal.lng, venuspostal.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8,
        
    ).add_to(vegan_map)
for lat, lng, name in zip(neighgeo['Latitude'], neighgeo['Longitude'], neighgeo['Neighbourhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='pink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.8,
        parse_html=False).add_to(vegan_map)
# display map
vegan_map

In [72]:
cdmg=clusteredmd.groupby('Labels').count()
cdmg

,Neighbourhood,Number of veg venu,Has veg venue?
Labels,,,
BadSpot,79,79,79
GoodSpot,13,13,13
OKspot,48,48,48


Vegan Spots map

In [74]:


# create a plain world map
vegan_spot_map = folium.Map(location=[43.653963, -79.387207], zoom_start=11, tiles='Mapbox Bright')
vegan_spot_map.choropleth(
    geo_data=geodata,
   data=df1,
   columns=['Neighbourhood Number','Labels'],
   key_on='properties.AREA_SHORT_CODE',
    fill_color='YlOrRd', 
    fill_opacity=0.7, 
    line_opacity=0.2,
    legend_name='Spots for a new vegan/vegetarian place 0-Bad Spot 1-OK Spot 2-Good'
)
for lat, lng, label in zip(venuspostal.lat, venuspostal.lng, venuspostal.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.8
    ).add_to(vegan_spot_map)
    
for lat, lng, name in zip(neighgeo['Latitude'], neighgeo['Longitude'], neighgeo['Neighbourhood']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='pink',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.3,
        parse_html=False).add_to(vegan_spot_map) 

# display map
vegan_spot_map


